# Textcuboid


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import re
import os
import string
import json
import keras
import wordninja
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
TRAIN_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/train"
TEST_DATA_PATH = "C:/Users/user/Desktop/bilm-tf-master/textdataset/IMDB/aclImdb/test"

def read_text_file(path):
    labels = ['neg','pos']
    if os.path.exists(path):
        text=[]
        text_label =[]
        for directory_name in os.listdir(path):
            if directory_name in labels:
                label_index = labels.index(directory_name)
                data_path = os.path.join(path,directory_name)
                for file in os.listdir(data_path):
                    with open(os.path.join(data_path,file),'r', encoding='utf-8') as f:
                        text.append(f.read())
                        text_label.append(label_index)
        return pd.DataFrame(text,columns =['texts']),pd.DataFrame(text_label,columns =['label'])
    
x_train,y_train = read_text_file(TRAIN_DATA_PATH) 
x_test,y_test = read_text_file(TEST_DATA_PATH) 

train = pd.concat([x_train, y_train], axis=1)
test = pd.concat([x_test, y_test], axis=1)

train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

train[['texts']] = train[['texts']].applymap(lambda x:remove_punctuation(x))
train[['texts']] = train[['texts']].applymap(lambda x:x.lower())
test[['texts']] = test[['texts']].applymap(lambda x:remove_punctuation(x))
test[['texts']] = test[['texts']].applymap(lambda x:x.lower())

def clean_text(text):
    text=str(text).lower() #Converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text=re.sub('\[.*?\]', '', text) #removes HTML tags
    text=re.sub('https?://\S+|www\.\S+', '', text) #removes url
    text=re.sub(r"["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", "", text) #removes emojis
    text=re.sub('[%s]' % re.escape(string.punctuation),'',text) #removes punctuations
    return text

X_train = train.drop(columns=['label'])
X_test = test.drop(columns=['label'])
y_train = train['label']
y_test = test['label']

X_train = X_train['texts'].apply(clean_text)
X_test = X_test['texts'].apply(clean_text)

pattern = '[^a-z ]'
Clean_X_train=[]
Clean_X_test=[]

for sen in X_train:
    Clean_X_train.append(re.sub(pattern, '', str(sen)))
    
for sen in X_test:
    Clean_X_test.append(re.sub(pattern, '', str(sen)))

y_train=list(y_train)
y_test=list(y_test)

train_df = pd.DataFrame({'X_train': Clean_X_train, 'y_train': y_train})
test_df = pd.DataFrame({'X_test': Clean_X_test, 'y_test': y_test})

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 8000개의 샘플을 랜덤하게 추출
train_df = train_df.groupby('y_train').apply(lambda x: x.sample(n=8000, random_state=42))

# 레이블 값에 따라 데이터프레임을 그룹화하고 각 그룹에서 2000개의 샘플을 랜덤하게 추출
test_df = test_df.groupby('y_test').apply(lambda x: x.sample(n=2000, random_state=42))

# 인덱스를 재설정합니다. drop=True 옵션을 사용하여 기존 인덱스를 제거합니다.
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

x_train = train_df['X_train'].tolist()
y_train = train_df['y_train'].tolist()
x_test = test_df['X_test'].tolist()
y_test = test_df['y_test'].tolist()

to_txt=x_train+x_test
y=y_train+y_test

In [3]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [4]:
#수동으로 cross validation을 하기위한 코드
cnt=5  #cnt가 2일때 두번째 시행
cnt=cnt-2

def exclude_list(input_list, cnt):
    return input_list[:cnt*2000]+input_list[cnt*2000+2000:8000]+input_list[8000:cnt*2000+8000]+input_list[cnt*2000+10000:]

if cnt>-1:
    x_train_cnt=exclude_list(x_train,cnt)+x_test
    y_train_cnt=exclude_list(y_train,cnt)+y_test
    x_test_cnt=x_train[cnt*2000:cnt*2000+2000]+x_train[cnt*2000+8000:cnt*2000+10000]
    y_test_cnt=y_train[cnt*2000:cnt*2000+2000]+y_train[cnt*2000+8000:cnt*2000+10000]

In [5]:
if cnt>-1:
    x_train=x_train_cnt
    y_train=y_train_cnt
    x_test=x_test_cnt
    y_test=y_test_cnt
    to_txt=x_train+x_test
    y=y_train+y_test

## 2) textcuboid 분류

In [ ]:
import random
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from keras.models import load_model

In [6]:
textcuboid=np.load('./1-Channel textcuboid_IMDB(bert).npy')

In [7]:
textcuboid_test=np.load('./1-Channel textcuboid_test_IMDB(bert).npy')

In [8]:
#Train 데이터 섞기

tmp = [[x,y] for x, y in zip(textcuboid, y_train)]
random.shuffle(tmp)
textcuboid = [n[0] for n in tmp]
y_train = [n[1] for n in tmp]
textcuboid=np.array(textcuboid)
y_train=np.array(y_train)

In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_{epoch}.h5", save_best_only=False, period=5
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=2, min_lr=0.0001
    ),
]

In [11]:
input_layer = Input(shape=(102, 768))
conv1 = Conv1D(1024, 1, padding='valid', activation='relu')(input_layer)
pooling = GlobalMaxPooling1D()(conv1)
pooling = Dropout(0.5)(pooling)
x = Dense(256, activation='relu')(pooling)
x = Dropout(0.5)(x)
output_layer = Dense(2, activation='softmax')(x)
model = Model(inputs=input_layer, outputs=output_layer)

In [12]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 102, 768)]        0         
                                                                 
 conv1d (Conv1D)             (None, 102, 1024)         787456    
                                                                 
 global_max_pooling1d (Globa  (None, 1024)             0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                             

In [14]:
x_train=textcuboid[1000:]
x_val=textcuboid[:1000]
y_train1=y_train[1000:]
y_val=y_train[:1000]

In [15]:
history = model.fit(x_train, y_train1,callbacks=callbacks, epochs=30,batch_size=256,validation_data=(x_val, y_val))

Epoch 1/30
59/59 [==============================] - 3s 29ms/step - loss: 2.3083 - accuracy: 0.5225 - val_loss: 0.6858 - val_accuracy: 0.5120 - lr: 0.0010
Epoch 2/30
59/59 [==============================] - 1s 24ms/step - loss: 0.6809 - accuracy: 0.5616 - val_loss: 0.6387 - val_accuracy: 0.8210 - lr: 0.0010
Epoch 3/30
59/59 [==============================] - 1s 25ms/step - loss: 0.5918 - accuracy: 0.6719 - val_loss: 0.3962 - val_accuracy: 0.8440 - lr: 0.0010
Epoch 4/30
59/59 [==============================] - 1s 24ms/step - loss: 0.4441 - accuracy: 0.7997 - val_loss: 0.3321 - val_accuracy: 0.8700 - lr: 0.0010
Epoch 5/30
59/59 [==============================] - 1s 25ms/step - loss: 0.3773 - accuracy: 0.8395 - val_loss: 0.3008 - val_accuracy: 0.8720 - lr: 0.0010
Epoch 6/30
59/59 [==============================] - 2s 26ms/step - loss: 0.3259 - accuracy: 0.8672 - val_loss: 0.2756 - val_accuracy: 0.8770 - lr: 0.0010
Epoch 7/30
59/59 [==============================] - 1s 25ms/step - loss: 0.2

In [16]:
y_test=np.array(y_test)

In [17]:
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 0s 4ms/step - loss: 0.4069 - accuracy: 0.8942


[0.4069397747516632, 0.8942499756813049]

In [21]:
model = load_model('best_model_10.h5')
model.evaluate(textcuboid_test,y_test)

125/125 [==============================] - 1s 4ms/step - loss: 0.2770 - accuracy: 0.8857


[0.27700716257095337, 0.8857499957084656]